### Setup Environment

In [1]:
import os
#os.environ["S3_ENDPOINT_URL"]="minio:9000"
#os.environ["AWS_ACCESS_KEY_ID"]="lwycuW6S5f7yJZt65tRK"
#os.environ["AWS_SECRET_ACCESS_KEY"]="d6hXquiXGpbmfR8OdX7Byd716hmhN87xTyCX8S0K"
os.environ["MLRUN_DBPATH"]="http://mlrun-api:8080"
import mlrun

In [2]:
# Set the environment:
project_name='mnist-training'

#mlrun.set_environment(env_file='mlrun.env', artifact_path=f's3://mlrun/projects/{project_name}/artifacts')
#mlrun.set_environment('http://mlrun-api:8080', env_file='mlrun.env', artifact_path='s3://mlrun')
mlrun.set_environment('http://mlrun-api:8080', artifact_path='./artifacts')

('default', '/home/jovyan/mnist-demo/artifacts')

### Create Project

In [3]:
# Create the project:
project_dir = os.path.abspath('./')
project = mlrun.get_or_create_project(project_name, project_dir, user_project=False)
print(project_dir)

> 2024-05-14 19:24:44,997 [debug] Project not found in db: {'project_name': 'mnist-training'}
> 2024-05-14 19:24:45,188 [info] Created and saved project: {'name': 'mnist-training', 'from_template': None, 'overwrite': False, 'context': '/home/jovyan/mnist-demo', 'save': True}
> 2024-05-14 19:24:45,189 [info] Project created successfully: {'project_name': 'mnist-training', 'stored_in_db': True}
/home/jovyan/mnist-demo


### Create Training Function

In [4]:
# Create the training function.
trainer = project.set_function(
    "mnist_training_with_mlrun.py", name="trainer", kind="job", 
    image="mlrun/mlrun", 
    requirements=["minio", "torch", "torchvision"],
    handler="train_model"
)
print(type(trainer))

<class 'mlrun.runtimes.kubejob.KubejobRuntime'>


### Run the function

In [5]:
# Sample hyperparameters
training_parameters = {
    'batch_size': 32,
    'device': 'cpu',
    'dropout_input': 0.2,
    'dropout_hidden': 0.5,
    'epochs': 2,
    'input_size': 784,
    'hidden_sizes': [1024, 1024, 1024, 1024],
    'lr': 0.025,
    'momentum': 0.5,
    'output_size': 10,
    'smoke_test_size': -1
    }

In [ ]:
# Run the function.
trainer_run = project.run_function(
    "trainer",
    inputs={"bucket_name": "mnist", "loader_type": "memory"},
    params={"training_parameters": training_parameters},
    local=True
)

> 2024-05-14 19:24:45,218 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-05-14 19:24:45,220 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-05-14 19:24:45,230 [info] Storing function: {'name': 'trainer-train-model', 'uid': 'b76ed4aeb74a44d49d8e55edef8f6bbf', 'db': 'http://mlrun-api:8080'}
> 2024-05-14 19:24:45,281 [debug] Starting local run: /home/jovyan/mnist-demo/mnist_training_with_mlrun.py # train_model
> 2024-05-14 19:24:47,883 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-05-14 19:24:47,884 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-05-14 19:24:47,885 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-05-

100%|██████████| 9912422/9912422 [00:00<00:00, 26728060.06it/s]


Extracting ./mnistdata/MNIST/raw/train-images-idx3-ubyte.gz to ./mnistdata/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1685717.98it/s]


Extracting ./mnistdata/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnistdata/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 10496371.74it/s]


Extracting ./mnistdata/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnistdata/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3531799.92it/s]

Extracting ./mnistdata/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnistdata/MNIST/raw

1487 2024-05-14 19:24:50,024 | INFO | Creating the model.
1487 2024-05-14 19:24:50,047 | INFO | Training the model.


Epoch 1/2:
Training: 100% |██████████| 1875/1875 [00:50<00:00, 36.80Batch/s, NLLLoss=0.11, accuracy=0.969]  
Validating: 100% |██████████| 313/313 [00:02<00:00, 105.64Batch/s, NLLLoss=0.115, accuracy=1]     

Summary:
+----------+--------------------+
| Metrics  |       Values       |
+----------+--------------------+
| NLLLoss  | 0.1542149782180786 |
| accuracy |      0.96875       |
+----------+--------------------+
> 2024-05-14 19:25:44,322 [debug] log artifact training_NLLLoss.html at /home/jovyan/mnist-demo/artifacts/trainer-train-model/0/training_NLLLoss.html.html, size: 3764000, db: Y
> 2024-05-14 19:25:44,389 [debug] log artifact training_accuracy.html at /home/jovyan/mnist-demo/artifacts/trainer-train-model/0/training_accuracy.html.html, size: 3740994, db: Y
> 2024-05-14 19:25:44,433 [debug] log artifact validation_NLLLoss.html at /home/jovyan/mnist-demo/artifacts/trainer-train-model/0/validation_NLLLoss.html.html, size: 3726722, db: Y
> 2024-05-14 19:25:44,476 [debug] log art

In [ ]:
import torch_utilities as tu

train_loader, test_loader, _ = tu.create_memory_data_loaders(32)
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    bra

In [ ]:
train_loader, test_loader, _ = tu.create_minio_data_loaders('mnist', 32)
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    bra